## Variational Quantum Deflation Tutorial

#### H2 example with overlap calculated classically. Work in progress for "swap test" quantum circuit for overlap computation.

In [1]:
import sys

sys.path.insert(0, '/Users/shijunang/qibochem/src')

In [2]:
import numpy as np

from qibo import Circuit, gates
from qibo.optimizers import optimize

In [3]:
# VQD Loss Function

def vqd_loss(params, circuit, hamiltonian, vqd_beta, vqe_gs):
    circuit.set_parameters(params)
    result = hamiltonian.backend.execute_circuit(circuit)
    final_state = result.state()
    overlap = np.inner(vqe_gs, final_state)
    # print(hamiltonian.expectation(final_state), overlap)
    return hamiltonian.expectation(final_state) + vqd_beta * np.abs(overlap) ** 2

In [4]:
import numpy as np
from qibo.models import VQE

from qibochem.driver import Molecule
from qibochem.ansatz import hf_circuit, ucc_circuit, ucc_ansatz, givens_excitation_ansatz

# Define the H2 molecule and obtain its 1-/2- electron integrals with PySCF
h2 = Molecule([('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.742))])
h2.run_pyscf()
# Generate the molecular Hamiltonian
hamiltonian = h2.hamiltonian()

# Build a UCC circuit ansatz for running VQE
circuit = ucc_ansatz(h2, excitations=[[0,1,2,3]])

# Create and run the VQE, starting with random initial parameters
vqe = VQE(circuit, hamiltonian)

initial_parameters = np.random.uniform(0.0, 2*np.pi, 8)
best, vqe_params, extra = vqe.minimize(initial_parameters)

[Qibo 0.2.16|INFO|2025-09-16 10:34:42]: Using numpy backend on /CPU:0


In [9]:
results = vqe.circuit()
vqe_gs = results.state()

In [10]:
givens_circuit = givens_excitation_ansatz(h2, excitations=[[1,2]], include_hf=False, use_mp2_guess=False)
givens_circuit.draw()

0: ───────────────
1: ─o─RY─X─RY─X─o─
2: ─X────o────o─X─
3: ───────────────


In [76]:
# Create a concatenated circuit of UCCSD and Given's

vqd_circuit = circuit + givens_circuit 
all_params = np.append(vqe_params, np.random.rand(len(givens_circuit.get_parameters())))
vqd_circuit.set_parameters(all_params)

for count in range(len(circuit.trainable_gates)):
    del vqd_circuit.trainable_gates[0]

# All trainable parameters (Given's trainable, UCCSD not trainable)
vqd_params = all_params[len(vqe_params):]

#nparams has to be manually updated
vqd_circuit.trainable_gates.nparams = len(vqd_circuit.trainable_gates)

vqd_circuit.trainable_gates.set = set(vqd_circuit.trainable_gates)

In [58]:
best, params, extra = optimize(vqd_loss, vqd_params, args=(vqd_circuit, hamiltonian, 100, vqe_gs), method='BFGS')

[Qibo 0.2.16|ERROR|2025-09-16 10:56:34]: Given list of parameters has length 10 while the circuit contains 2 parametrized gates.


ValueError: Given list of parameters has length 10 while the circuit contains 2 parametrized gates.

In [13]:
from qibochem.measurement import expectation

vqd_circuit.set_parameters(params)
print(f"Expectation value: {expectation(vqd_circuit, hamiltonian):.8f} (Exact first excited state of H2)")

Expectation value: -0.53320754 (Exact first excited state of H2)
